# Machine learning on wine

**Topics:** Text analysis, linear regression, logistic regression, text analysis, classification

**Datasets**

- **wine-reviews.csv** Wine reviews scraped from https://www.winemag.com/
- **Data dictionary:** just go [here](https://www.winemag.com/buying-guide/tenuta-dellornellaia-2007-masseto-merlot-toscana/) and look at the page

## The background

You work in the **worst newsroom in the world**, and you've had a hard few weeks at work - a couple stories killed, a few scoops stolen out from under you. It's not going well.

And because things just can't get any worse: your boss shows up, carrying a huge binder. She slams it down on your desk.

"You know some machine learning stuff, right?"

You say "no," but she isn't listening. She's giving you an assignment, the _worst assignment_...

> Machine learning is the new maps. Let's get some hits!
>
> **Do some machine learning on this stuff.**

"This stuff" is wine reviews.

## A tiny, meager bit of help

You have a dataset. It has some stuff in it:

* **Numbers:**
    - Year published
    - Alcohol percentage
    - Price
    - Score
    - Bottle size
* **Categories:**
    - Red vs white
    - Different countries
    - Importer
    - Designation
    - Taster
    - Variety
    - Winery
* **Free text:**
    - Wine description

In [6]:
import pandas as pd
import re

# Cleaning up your data

Many of these pieces - the alcohol, the year produced, the bottle size, the country the wine is from - aren't in a format you can use. Convert the ones to numbers that are numbers, and extract the others from the appropriate strings.

In [127]:
df = pd.read_csv('wine-reviews.csv')
df.head(5)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating
0,https://www.winemag.com/buying-guide/artadi-20...,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black pl...",Michael Schachner,"$25, Buy Now",Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,14.5%,750 ml,Red,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review]
1,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2012 Stoller Vineyard Chardonnay (Du...,"A tiny production wine, this is rich, tart and...",Paul Gregutt,"$65, Buy Now",Stoller Vineyard,Chardonnay,"Dundee Hills, Willamette Valley, Oregon, US",Adelsheim,13.5%,750 ml,White,NaN,12/1/2014,Not rated yet [Add Your Review]
2,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2013 Ribbon Springs Vineyard Auxerro...,This is another fine vintage for this rare win...,Paul Gregutt,"$25, Buy Now",Ribbon Springs Vineyard,"Auxerrois, Other White","Ribbon Ridge, Willamette Valley, Oregon, US",Adelsheim,13.5%,750 ml,White,NaN,12/1/2014,Not rated yet [Add Your Review]
3,https://www.winemag.com/buying-guide/jcb-2011-...,90.0,JCB 2011 No. 11 Pinot Noir (Sonoma Coast),Light in color and lilting floral aromas of ro...,Virginie Boone,"$65, Buy Now",No. 11,Pinot Noir,"Sonoma Coast, Sonoma, California, US",JCB,13%,750 ml,Red,NaN,12/1/2014,Not rated yet [Add Your Review]
4,https://www.winemag.com/buying-guide/pazo-pond...,90.0,Pazo Pondal 2013 Albariño (Rías Baixas),"Alluring, inviting aromas of white flowers, me...",Michael Schachner,"$17, Buy Now",NaN,Albariño,"Rías Baixas, Galicia, Spain",Pazo Pondal,13%,750 ml,White,Vinaio Imports,12/1/2014,Not rated yet [Add Your Review]


In [143]:
df.dtypes

url                        object
wine_points               float64
wine_name                  object
wine_desc                  object
taster                     object
price                      object
designation                object
variety                    object
appellation                object
winery                     object
alcohol                   float64
bottle size               float64
category                   object
importer                   object
date published     datetime64[ns]
user avg rating            object
dtype: object

In [128]:
df['alcohol'] = df['alcohol'].str.replace(r'%', r'').astype('float')
df['date published'] =  pd.to_datetime(df['date published'], format='%m/%d/%Y')
df["appellation"] = df["appellation"].str.split().str[-1]

In [152]:
df['wine_points'] = df['wine_points'].astype('int')

In [129]:
df['bottle size'] =  [re.sub(r'ml','', str(x)) for x in df['bottle size']]
df['bottle size'] =  [re.sub(r'ML','', str(x)) for x in df['bottle size']]
df['bottle size'] =  [re.sub(r' ml','', str(x)) for x in df['bottle size']]
df['bottle size'] =  [re.sub(r'L','', str(x)) for x in df['bottle size']]
df['bottle size'] =  [re.sub(r' L','', str(x)) for x in df['bottle size']]
df['bottle size'] = df['bottle size'].astype('float')

In [130]:
df.price.value_counts()

N/A,  Buy Now       2994
$20,  Buy Now       1981
$25,  Buy Now       1712
$30,  Buy Now       1683
$15,  Buy Now       1657
                    ... 
$26.99,  Buy Now       1
$675,  Buy Now         1
$188,  Buy Now         1
$44.99,  Buy Now       1
$510,  Buy Now         1
Name: price, Length: 313, dtype: int64

In [142]:
df['price'] =  [re.sub(r'Buy Now','', x) for x in df['price']]
df['price'] =  [re.sub(r'\$','', x) for x in df['price']]
df['price'] =  [re.sub(r',','', x) for x in df['price']]
df['price']

0         25  
1         65  
2         25  
3         65  
4         17  
         ...  
42290     24  
42291     22  
42292     10  
42293     17  
42294    N/A  
Name: price, Length: 42295, dtype: object

In [154]:
df.dtypes

url                        object
wine_points                 int64
wine_name                  object
wine_desc                  object
taster                     object
price                      object
designation                object
variety                    object
appellation                object
winery                     object
alcohol                   float64
bottle size               float64
category                   object
importer                   object
date published     datetime64[ns]
user avg rating            object
dtype: object

## What might be interesting in this dataset?

Maybe start out playing around _without_ machine learning. Here are some thoughts to get you started:

* I've heard that since the 90's wine has gone through [Parkerization](https://www.estatewinebrokers.com/blog/the-parkerization-of-wine-in-the-1990s-and-beyond/), an increase in production of high-alcohol, fruity red wines thanks to the influence of wine critic Robert Parker.
* Red and white wines taste different, obviously, but people always use [goofy words to describe them](https://winefolly.com/tutorial/40-wine-descriptions/)
* Once upon a time in 1976 [California wines proved themselves against France](https://en.wikipedia.org/wiki/Judgment_of_Paris_(wine)) and France got very angry about it

In [148]:
df.groupby('category').alcohol.value_counts(ascending=True)

category  alcohol
Dessert   2.0           1
          8.8           1
          8.9           1
          10.2          1
          10.4          1
                     ... 
White     14.0        566
          12.0        687
          12.5       1413
          13.5       1814
          13.0       1953
Name: alcohol, Length: 417, dtype: int64

In [151]:
df.groupby('wine_points').appellation.value_counts(ascending=True)

wine_points  appellation
80.0         Germany        1
             Portugal       1
             France         2
             Romania        2
             Australia      4
                           ..
100.0        Portugal       1
             Spain          1
             France         2
             US             3
             Austria        4
Name: appellation, Length: 371, dtype: int64

## But machine learning?

Well, you can usually break machine learning down into a few different things. These aren't necessarily perfect ways of categorizing things, but eh, close enough.

* **Predicting a number**
    - Linear regression
    - For example, how does a change in unemployment translate into a change in life expectancy?
* **Predicting a category** (aka classification)
    - Lots of algos options: logistic regression, random forest, etc
    - For example, predicting cuisines based on ingredients
* **Seeing what influences a numeric outcome**
    - Linear regression since the output is a number
    - For example, minority and poverty status on test scores 
* **Seeing what influences a categorical outcome**
    - Logistic regression since the output is a category
    - Race and car speed for if you get a waring vs ticket
    - Wet/dry pavement and car weight if you survive or not in a car crash)

We have numbers, we have categories, we have all sorts of stuff. **What are some ways we can mash them together and use machine learning?**

### Brainstorm some ideas

Use the categories above to try to come up with some ideas. Be sure to scroll up where I break down categories vs numbers vs text!

**I'll give you one idea for free:** if you don't have any ideas, start off by creating a classifier that determines whether a wine is white or red based on the wine's description.

You can also go to https://library.columbia.edu and see if you can find some academic papers about wine. I'm sure they'll inspire you! (and they might even have some ML ideas in them you can steal, too)

My ideas:

- A classifier that determines wine_points based on price

- A classifier that determines if a wine is red or white based on alcohol percentage

- A classifier that determines wine_points based on the description

In [158]:
df.head(2)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating,is_red
0,https://www.winemag.com/buying-guide/artadi-20...,90,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black pl...",Michael Schachner,25,Viñas de Gain,Tempranillo,Spain,Artadi,14.5,750.0,Red,Folio Fine Wine Partners,2014-12-01,Not rated yet [Add Your Review],0
1,https://www.winemag.com/buying-guide/adelsheim...,90,Adelsheim 2012 Stoller Vineyard Chardonnay (Du...,"A tiny production wine, this is rich, tart and...",Paul Gregutt,65,Stoller Vineyard,Chardonnay,US,Adelsheim,13.5,750.0,White,NaN,2014-12-01,Not rated yet [Add Your Review],0


In [161]:
df.wine_desc.dropna()

0        Inky, minerally aromas of blackberry, black pl...
1        A tiny production wine, this is rich, tart and...
2        This is another fine vintage for this rare win...
3        Light in color and lilting floral aromas of ro...
4        Alluring, inviting aromas of white flowers, me...
                               ...                        
42290    Very fruit forward in cherries and pomegranate...
42291    Very dry and robust in the mouth, a clean wine...
42292    Black in color and saturated with plum, fruit ...
42293    Muddled to start, with chocolate and earth aro...
42294    Smells oaky-beetrooty. Cherries, cocoa, nut an...
Name: wine_desc, Length: 42039, dtype: object

In [174]:
df.wine_desc = df.wine_desc.astype('U')

# Implement 2 of your machine learning ideas

#### a classifier that determines whether a wine is white or red based on the wine's description.

In [162]:
df['is_red'] = (df.category == 'Red').astype(int)
df.is_red.value_counts()

1    25392
0    16903
Name: is_red, dtype: int64

In [176]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Make a vectorizer
vectorizer = TfidfVectorizer()

# Learn and count the words in df.content
matrix = vectorizer.fit_transform(df.wine_desc)

# Convert the matrix of counts to a dataframe
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names())
words_df.head()

,000,002,01,01s,02,02s,03,030,03s,04,...,½seasoningï,½t,àmaurice,élevage,émilion,épernay,étalon,über,überbest,ürziger
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
X = words_df
y = df.is_red

In [179]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [180]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [181]:
clf.score(X_test, y_test)

0.9600907887270664

In [182]:
df.is_red.value_counts(normalize=True)

1    0.600355
0    0.399645
Name: is_red, dtype: float64

In [183]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not red', 'red'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted not red,Predicted red
Is not red,3893,347
Is red,75,6259


In [184]:
import eli5

eli5.show_weights(clf, feature_names=vectorizer.get_feature_names())

Weight,Feature
0.0370 ± 0.0780,tannins
0.0355 ± 0.0715,cherry
0.0250 ± 0.0623,apple
0.0229 ± 0.0559,black
0.0182 ± 0.0452,peach
0.0181 ± 0.0360,citrus
0.0148 ± 0.0368,blackberry
0.0143 ± 0.0358,lemon
0.0135 ± 0.0343,pear
0.0120 ± 0.0263,white


#### a classifier that determines determines if wine_points are 90 or higher based on the description

In [185]:
df['is_high'] = (df.wine_points > 89).astype(int)
df.is_high.value_counts()

0    22312
1    19983
Name: is_high, dtype: int64

In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Make a vectorizer
vectorizer = TfidfVectorizer()

# Learn and count the words in df.content
matrix = vectorizer.fit_transform(df.wine_desc)

# Convert the matrix of counts to a dataframe
words2_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names())
words2_df.head()

,000,002,01,01s,02,02s,03,030,03s,04,...,½seasoningï,½t,àmaurice,élevage,émilion,épernay,étalon,über,überbest,ürziger
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [188]:
X = words2_df
y = df.is_high

In [189]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [190]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [191]:
clf.score(X_test, y_test)

0.8072630981653112

In [192]:
df.is_high.value_counts(normalize=True)

0    0.527533
1    0.472467
Name: is_high, dtype: float64

In [193]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not high', 'high'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

#label names are wrong -- it should say "predicted not high" and "predicted high"

,Predicted not red,Predicted red
Is not red,4692,828
Is red,1210,3844


In [194]:
import eli5

eli5.show_weights(clf, feature_names=vectorizer.get_feature_names())

Weight,Feature
0.0136 ± 0.0224,this
0.0118 ± 0.0168,flavors
0.0086 ± 0.0076,the
0.0083 ± 0.0127,rich
0.0080 ± 0.0035,and
0.0078 ± 0.0058,of
0.0077 ± 0.0087,with
0.0071 ± 0.0136,from
0.0064 ± 0.0059,wine
0.0063 ± 0.0088,finish
